<a href="https://colab.research.google.com/github/zzhenxi/AIB-Jeju-weather-api-service/blob/main/jh_%EC%8B%9C%EA%B0%81%ED%99%94(%EC%9E%91%EC%97%85%EC%A4%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업 목록 

1.  **셀럽이 어떻게 활동해야 팬들이 좋아하는지 분석**
    - 상위 n퍼센트에 속하는 셀럽에 대한 분석과 시각화 자료
        
        `그래프로 (직관적으로) 보여주는 것에 중점`
        
        - 셀럽이 먼저 대화를 시작한 경우 비율 (날짜별로)
        - 셀럽의 태도에 따른 시간의 흐름에 따른 긍정도 변화의 시각화
        - 시간의 흐름에 따른 셀럽의 말에 대한 팬의 응답률
            
            → 이렇게 했더니 팬의 응답률이 올라갔다고 제시하기 위함
            
        - 시간의 흐름에 따른 긍정도 변화 선그래프
            
             → 이렇게 했더니 팬의 긍정도가 올라갔다고 제시하기 위함

## 데이터 가져오기
- data : 모든 데이터 목록

In [1]:
import pandas as pd 
import numpy as np
import matplotlib as plt

In [2]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv("/content/drive/MyDrive/AIB_PJ2/dataset/fancimm_data.csv")

In [4]:
data

,date,time,celeb,code,comments,file,label
0,12-Jul-21,20:39,VAYA,KAMF,님이 참여했습니다.,NaN,1.0
1,12-Jul-21,20:40,VAYA,KAMF,사랑해요🖤,NaN,2.0
2,12-Jul-21,20:40,VAYA,MLSC,님이 참여했습니다.,NaN,1.0
3,12-Jul-21,20:41,VAYA,ZSOS,님이 참여했습니다.,NaN,1.0
4,12-Jul-21,20:41,VAYA,MLSC,단톡 전용 인스타 대체일지... 자려고했는데 파들..,NaN,1.0
...,...,...,...,...,...,...,...
10324,18-Aug-21,8:56,JOPU,BYZG,ㅎㅎ 오늘도 비가 주륵주륵~ 벼보리님 우산 잘 쓰고 다니셔요 !,NaN,1.0
10325,18-Aug-21,17:38,JOPU,JOPU,Operator오늘은 ~ 수요일 ~ 스팀겜 하는 날~ 일주일의 절반이 벌써 다가왔네...,NaN,1.0
10326,18-Aug-21,18:03,JOPU,CCAA,재밌는시간 보내셨겠네요 ㅎㅎ\n저는 학교갔는데 생각했던것보단 정겹고 좋네요 은근 ㅎ...,NaN,1.0
10327,18-Aug-21,18:15,JOPU,ZWYM,보리쟝이무화과를 먹었으니 저는 복숭아를 먹겠읍니다 후후\n푹 쉬고 7시에 방송에서...,NaN,1.0


In [5]:
data_info = pd.read_excel("/content/drive/MyDrive/AIB_PJ2/dataset/계정 정보 및 설문조사.xlsx", sheet_name="계정 정보", keep_default_na=False)

In [6]:
data_surv = pd.read_excel("/content/drive/MyDrive/AIB_PJ2/dataset/계정 정보 및 설문조사.xlsx", sheet_name="설문조사", keep_default_na=False)

## 데이터 전처리 

### data_info   
- data_info -> celeb_li, user_li

In [7]:
data_info

,셀럽 계정,테스트 계정
0,VAYA,BYZG
1,WKDG,XPBD
2,XMSQ,YQVX
3,CJSL,YSUL
4,DQZB,RLGI
5,JLDN,QOGI
6,OSQR,IYWE
7,JSEU,MDYT
8,LOVM,SLYG
9,JOPU,SNLX


In [8]:
# 각각 계정 리스트 안에 넣기 
celeb_li = list(np.array(data_info['셀럽 계정'].to_list()))
celeb_li = [id for id in celeb_li if id != ''] # 공백 없애기, user_li의 경우 공백은 없을 확률이 높으므로 따로 코드 추가 x

user_li = list(np.array(data_info['테스트 계정'].to_list()))


### data_surv
- 추천점수 부분이 없는, 공백(null)값을 포함한 행 제거
- 셀럽별 / 유저별 survey 분리
- data_surv_score : 추천 점수가 있는 항목
- data_surv_pay : 얼마 낼 의향인지 가격이 있는 항목

In [9]:
data_surv.columns

Index(['코드', '추천 점수', '점수 이유', '지불 의사 금액', '플랫폼 수수료', '추가 요청 기능', '기타 의견'], dtype='object')

In [10]:
data_surv_score = data_surv[data_surv['추천 점수'] != '']

In [11]:
data_surv_score.head()

,코드,추천 점수,점수 이유,지불 의사 금액,플랫폼 수수료,추가 요청 기능,기타 의견
0,KAMF,10,만족해서,"월 10,000원 이상 ~30,000원 미만",5% 이상 ~ 10% 미만,톡 알림 끄고 키기,넹
1,MLSC,8,셀럽과 대화가 가능한데 단톡방과는 다른 개별 대화같은느낌. 서로 부담없이 가능한 대화.,"월 30,000원 이상 ~50,000원 미만",5% 이상 ~ 10% 미만,인스타그램 피드처럼 셀럽에게만 보이는 포스트기능이 있으면 좋겠네요.,"앱이 많이 느려서 실행하면 기다림이 잦고, 셀럽이 메시지를 보내도 알림이 안 떠요...."
2,ZSOS,7,1대1처럼 대화를 하는 점이 좋으나 아직 오류가 많기 때문에,"월 3,000원 미만",5% 미만,캡쳐 기능이 있었으면 좋겠습니다,대화내용이 사라지거나 알람이 늦게 오는 오류들만 없어지면 좋겠습니다
3,GUKR,9,메시지가 셀럽에게만 보여져서 좋고 톡방에 여러명이 있어도 귓속말 답변도 받을 수 있...,"월 3,000원 미만",5% 미만,카톡처럼 이모티콘 사용이 가능했으면 좋겠어요,셀랍 방에 대화 내용을 살펴보려고 내리면 대화내용이 안 보이는 경우가 있어요
4,VAYA,7,현재는 불안정한 요소들이 많아서 더 개선되면 좋다고 생각합니다,"월 3,000원 미만",5% 이상 ~ 10% 미만,일단은 안정성만 더 좋아도 될 것 같아오,팬들과 소통을 중요하게 생각하는 셀럽들에겐 되게 좋은 어플


In [12]:
data_surv_pay = data_surv[data_surv['지불 의사 금액'] != '']

In [13]:
data_surv_pay.head()

,코드,추천 점수,점수 이유,지불 의사 금액,플랫폼 수수료,추가 요청 기능,기타 의견
0,KAMF,10,만족해서,"월 10,000원 이상 ~30,000원 미만",5% 이상 ~ 10% 미만,톡 알림 끄고 키기,넹
1,MLSC,8,셀럽과 대화가 가능한데 단톡방과는 다른 개별 대화같은느낌. 서로 부담없이 가능한 대화.,"월 30,000원 이상 ~50,000원 미만",5% 이상 ~ 10% 미만,인스타그램 피드처럼 셀럽에게만 보이는 포스트기능이 있으면 좋겠네요.,"앱이 많이 느려서 실행하면 기다림이 잦고, 셀럽이 메시지를 보내도 알림이 안 떠요...."
2,ZSOS,7,1대1처럼 대화를 하는 점이 좋으나 아직 오류가 많기 때문에,"월 3,000원 미만",5% 미만,캡쳐 기능이 있었으면 좋겠습니다,대화내용이 사라지거나 알람이 늦게 오는 오류들만 없어지면 좋겠습니다
3,GUKR,9,메시지가 셀럽에게만 보여져서 좋고 톡방에 여러명이 있어도 귓속말 답변도 받을 수 있...,"월 3,000원 미만",5% 미만,카톡처럼 이모티콘 사용이 가능했으면 좋겠어요,셀랍 방에 대화 내용을 살펴보려고 내리면 대화내용이 안 보이는 경우가 있어요
4,VAYA,7,현재는 불안정한 요소들이 많아서 더 개선되면 좋다고 생각합니다,"월 3,000원 미만",5% 이상 ~ 10% 미만,일단은 안정성만 더 좋아도 될 것 같아오,팬들과 소통을 중요하게 생각하는 셀럽들에겐 되게 좋은 어플


## 상위 n 퍼센트의 셀럽 목록, 팬의 숫자 
- 상위 n개의 셀럽 정하는 기준 
- 대화방에서 팬의 수 방의 참여자 만족도가 높은 순(셀럽, 팬 합쳐서) / 방의 활성화 순서 (인당 대화가 많은 순서) (전체 대화 / 참여자수) / 방의 부정도가 가장 낮은

### 1. 참여도가 높은 방 순서

In [14]:
### 우선 임시로 상위 10개
### 나중에 데이터 추가되면,, 상위 n퍼센트로 바꾸거나 해도 좋을듯 

In [15]:
data

,date,time,celeb,code,comments,file,label
0,12-Jul-21,20:39,VAYA,KAMF,님이 참여했습니다.,NaN,1.0
1,12-Jul-21,20:40,VAYA,KAMF,사랑해요🖤,NaN,2.0
2,12-Jul-21,20:40,VAYA,MLSC,님이 참여했습니다.,NaN,1.0
3,12-Jul-21,20:41,VAYA,ZSOS,님이 참여했습니다.,NaN,1.0
4,12-Jul-21,20:41,VAYA,MLSC,단톡 전용 인스타 대체일지... 자려고했는데 파들..,NaN,1.0
...,...,...,...,...,...,...,...
10324,18-Aug-21,8:56,JOPU,BYZG,ㅎㅎ 오늘도 비가 주륵주륵~ 벼보리님 우산 잘 쓰고 다니셔요 !,NaN,1.0
10325,18-Aug-21,17:38,JOPU,JOPU,Operator오늘은 ~ 수요일 ~ 스팀겜 하는 날~ 일주일의 절반이 벌써 다가왔네...,NaN,1.0
10326,18-Aug-21,18:03,JOPU,CCAA,재밌는시간 보내셨겠네요 ㅎㅎ\n저는 학교갔는데 생각했던것보단 정겹고 좋네요 은근 ㅎ...,NaN,1.0
10327,18-Aug-21,18:15,JOPU,ZWYM,보리쟝이무화과를 먹었으니 저는 복숭아를 먹겠읍니다 후후\n푹 쉬고 7시에 방송에서...,NaN,1.0


In [16]:
# 대화방 참여자가 총 몇명인지 어떻게 알아낼까? -> celeb별 대화 참여자 unique값 찾기, 총 참여자 수
data.groupby('celeb')

In [17]:
# celeb unique 값 찾기 # 따로 받은 celeb_li와 순서와 내용이 같음.
# a = list(data['celeb'].unique())
# 따라서 굳이 따로 구할 필요 없음 -> 이건 나중에 필요할때 다시 활용

In [18]:
# 셀럽별 대화방 참여자 총 몇명 (unique)
user_dic = {} # [celeb : 유저들 unique 값 리스트]
for id in celeb_li:
  val = list(data[data['celeb'] == id]['code'].unique())
  user_dic[id] = val

In [19]:
# 셀럽별 대화 총 몇개 
com_dic = {} # [celeb : 유저들 comments 개수]
for id in celeb_li:
  val = len(data[data['celeb'] == id]['comments'])
  com_dic[id] = val

In [20]:
# 셀럽별 대화방 활성화 정도
act_dic = {} # 얼마나 acitve 한지 
for id in celeb_li:
  user_num = len(user_dic[id]) # user_num : 유저 몇명?
  com_num = com_dic[id] # com_num : comments 몇개?
  act_dic[id] = round(com_num/user_num, 3) # 유저 / 총 대화 수로 acitve한 정도를 구해줍니다.

In [21]:
# act_dic을 바탕으로 상위 n 퍼센트를 구합니다. 
act_dic

{'CJSL': 31.426,
 'DQZB': 10.467,
 'JLDN': 26.143,
 'JOPU': 16.875,
 'JSEU': 4.9,
 'LOVM': 17.6,
 'OSQR': 5.867,
 'VAYA': 126.792,
 'WKDG': 51.97,
 'XMSQ': 73.71}

In [22]:
# dictionary active가 높은 순서대로 만들기 
sorted_dict = {}
for k, v in sorted(act_dic.items(), key = lambda item: item[1], reverse = True):
  sorted_dict[k] = v

# 코드 참고 https://codechacha.com/ko/python-sorting-dict/

In [23]:
# top 10 celeb list
top10_li = list(sorted_dict.keys())[:10]

In [24]:
top10_li

['VAYA',
 'XMSQ',
 'WKDG',
 'CJSL',
 'JLDN',
 'LOVM',
 'JOPU',
 'DQZB',
 'OSQR',
 'JSEU']

### 만약 데이터 전처리가 잘 되었다면 이 부분은 건너뛰어도 됩니다. 

In [25]:
data

,date,time,celeb,code,comments,file,label
0,12-Jul-21,20:39,VAYA,KAMF,님이 참여했습니다.,NaN,1.0
1,12-Jul-21,20:40,VAYA,KAMF,사랑해요🖤,NaN,2.0
2,12-Jul-21,20:40,VAYA,MLSC,님이 참여했습니다.,NaN,1.0
3,12-Jul-21,20:41,VAYA,ZSOS,님이 참여했습니다.,NaN,1.0
4,12-Jul-21,20:41,VAYA,MLSC,단톡 전용 인스타 대체일지... 자려고했는데 파들..,NaN,1.0
...,...,...,...,...,...,...,...
10324,18-Aug-21,8:56,JOPU,BYZG,ㅎㅎ 오늘도 비가 주륵주륵~ 벼보리님 우산 잘 쓰고 다니셔요 !,NaN,1.0
10325,18-Aug-21,17:38,JOPU,JOPU,Operator오늘은 ~ 수요일 ~ 스팀겜 하는 날~ 일주일의 절반이 벌써 다가왔네...,NaN,1.0
10326,18-Aug-21,18:03,JOPU,CCAA,재밌는시간 보내셨겠네요 ㅎㅎ\n저는 학교갔는데 생각했던것보단 정겹고 좋네요 은근 ㅎ...,NaN,1.0
10327,18-Aug-21,18:15,JOPU,ZWYM,보리쟝이무화과를 먹었으니 저는 복숭아를 먹겠읍니다 후후\n푹 쉬고 7시에 방송에서...,NaN,1.0


In [26]:
data['label'] = data['label'].dropna()

In [27]:
data['label'].value_counts()

1.0    7896
2.0    2340
0.0      90
Name: label, dtype: int64

In [30]:
# data['label'] = data['label'].astype(int)

ValueError: ignored

ValueError: Cannot convert non-finite values (NA or inf) to integer 가 발생

fillna로 해결

In [31]:
data['label'] = data['label'].fillna(9) # 우선 9로 na값을 대체 
index1 = data[data['label']==9].index # label 값이 9인 행 index를 추출
data = data.drop(index1) # 해당하는 index를 drop

In [37]:
# 마지막으로 label 행을 int로 바꿔준다.
data['label'] = data['label'].astype(int)

### 2. 부정도가 낮은 방 순서

In [122]:
a = data.groupby(['celeb', 'label']).count()

In [123]:
# data_012는 셀럽에 따라 0이 몇개, 1이 몇개, 2가 몇개인지 정리해놓은 dataframe
data_012 = a[['date']]

In [124]:
data_012

date
celeb label      
CJSL  0        11
      1      1565
      2       121
DQZB  0         4
      1       294
      2        16
JLDN  0         7
      1       811
      2        96
JOPU  0         2
      1       116
      2        17
JSEU  0         9
      1        37
      2         3
LOVM  1        86
      2         2
OSQR  0         4
      1        77
      2         7
VAYA  0        28
      1      1379
      2      1636
WKDG  0        17
      1      1464
      2       233
XMSQ  0         8
      1      2067
      2       209

In [53]:
list(data_012.loc['CJSL'].index)

[0, 1, 2]

In [129]:
# pivot_table로 보기 편하게 바꿔주기 / 0이 없는 곳은 0으로 자리 자동으로 메꿔줌
data_012 = pd.pivot_table(data_012,
                       index='celeb',
                       columns='label')

In [132]:
# 멀티 인덱스 해제 - 컬럼명 수정
data_012.columns = [0, 1, 2]

In [133]:
data_012

,0,1,2
celeb,,,
CJSL,11.0,1565.0,121.0
DQZB,4.0,294.0,16.0
JLDN,7.0,811.0,96.0
JOPU,2.0,116.0,17.0
JSEU,9.0,37.0,3.0
LOVM,0.0,86.0,2.0
OSQR,4.0,77.0,7.0
VAYA,28.0,1379.0,1636.0
WKDG,17.0,1464.0,233.0


In [ ]:
# 해야하는것 - 총합 컬럼 만들기, 부정도 컬럼 만들기 : 총합/0

### 3. 참여자 만족도가 높은 순서

### 1, 2, 3번 조합

### 셀럽이 먼저 대화를 시작한 경우 비율 (날짜별로) 아래꺼랑 합친 그래프면 좋을듯

### 셀럽의 태도에 따른 시간의 흐름에 따른 긍정도 변화의 시각화

### 시간의 흐름에 따른 셀럽의 말에 대한 팬의 응답률

### 시간의 흐름에 따른 긍정도 변화 선그래프

## 상위 n퍼센트 외 나머지

### 얼마를 낼 의향있소!의 비율과 만족도 (평균 점수, 점수 비율)